In [ ]:
import pandas as pd
import numpy as np
import os

### Notes
* Need to get location info for metro stations
* Need way to distribute population appropriately at metro stations

In [ ]:
total_homeless = 3573
total_metro = total_homeless // 2
total_camp = total_homeless - total_metro

In [ ]:
dir = "/Users/mohamaddalati/Desktop/Archive-Fall2022/MGSC-662/Project/MGSC 662 - Project/Data prep/Data_Prep.ipynb"
os.chdir(dir)
dft = pd.read_excel("Model_Data.xlsx", sheet_name="Transitional")
dfe = pd.read_excel("Model_Data.xlsx", sheet_name="Emergency")
dfh = pd.read_excel("Model_Data.xlsx", sheet_name="HotSpots")
dfn = pd.read_excel("Model_Data.xlsx", sheet_name="NewLocations")

### Transitional

In [ ]:
dft_piv = pd.pivot_table(dft, values="Beds", index="Index", aggfunc=np.sum)
dft_piv.columns = ["Total Beds"]

In [ ]:
dft = pd.merge(dft, dft_piv, how = "left", left_on="Index", right_on="Index")

In [ ]:
# Should we remove rows with empty costs or replace with the median value?
dft['Annual Cost'] = dft['Annual Cost'].replace({np.nan:dft['Annual Cost'].median()})

In [ ]:
dft['Adjusted Costs'] = (dft['Beds'] / dft['Total Beds']) * dft['Annual Cost']

In [ ]:
var_cost = 25860
dft['Variable Cost'] = dft['Beds'] * var_cost

In [ ]:
dft = dft.drop(columns="Index")

In [ ]:
dft.head()

### Emergency

In [ ]:
dfe_piv = pd.pivot_table(dfe, values="Beds", index="Index", aggfunc=np.sum)
dfe_piv.columns = ["Total Beds"]

In [ ]:
dfe = pd.merge(dfe, dfe_piv, how = "left", left_on="Index", right_on="Index")

In [ ]:
# Should we remove rows with empty costs or replace with the median value?
dfe['Annual Cost'] = dfe['Annual Cost'].replace({np.nan:dfe['Annual Cost'].median()})

In [ ]:
dfe['Adjusted Costs'] = (dfe['Beds'] / dfe['Total Beds']) * dfe['Annual Cost']

In [ ]:
var_cost = 25860
dfe['Variable Cost'] = dfe['Beds'] * var_cost

In [ ]:
dfe = dfe.drop(columns="Index")

In [ ]:
dfe.head()

### Combine Transitional and Emergency

In [ ]:
dfc = pd.concat([dft, dfe])
dfc.head()

### Hotspots

Should we use the metro stations and scale or use the camps

In [ ]:
dfh_camp = dfh[pd.isna(dfh['Density'])]
dfh_camp = dfh_camp[['Station', 'Density', 'Latitude', 'Longitude']]
dfh_metro = dfh[pd.notna(dfh['Density'])]
dfh_metro = dfh_metro[['Station', 'Density', 'Latitude', 'Longitude']]

In [ ]:
num_camps = dfh_camp.shape[0]
dfh_camp['Homeless'] = total_camp // num_camps

In [ ]:
# find the minimum density value
# put density of metro station divided by minimum density
low = dfh_metro['Density'].min()
dfh_metro['Scale'] = dfh_metro['Density'] / low
total = dfh_metro['Scale'].sum()
dfh_metro['Homeless'] = dfh_metro['Scale'] / total * total_metro
dfh_metro['Homeless'] = dfh_metro['Homeless'].round()

In [ ]:
dfh_metro = dfh_metro.drop(columns=['Density', 'Scale'])

In [ ]:
dfh_camp = dfh_camp.drop(columns=['Density'])

In [ ]:
dfd = pd.concat([dfh_metro, dfh_camp])

### Potential

In [ ]:
dfn.head()

### Export to Excel

In [ ]:
with pd.ExcelWriter('Final_Model_Data.xlsx') as writer:
    dft.to_excel(writer, sheet_name="Transitional", index=False)
    dfe.to_excel(writer, sheet_name="Emergency", index=False)
    dfc.to_excel(writer, sheet_name="CombinedHousing", index=False)
    dfh_camp.to_excel(writer, sheet_name="HotspotCamps", index=False)
    dfh_metro.to_excel(writer, sheet_name="HotspotsMetro", index=False)
    dfd.to_excel(writer, sheet_name="HotspotsAll", index=False)
    dfn.to_excel(writer, sheet_name="NewLocations", index=False)